In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pydot
import graphviz
from clr_callback import CyclicLR

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.client import device_lib 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import save_model, load_model

tf.random.set_seed(42)

train_directory = "./data/organized/train/"
val_directory = "./data/organized/val/"
test_directory = "./data/organized/test/"

In [2]:
# Allow Tensorflow to allocate GPU memory as needed, rather than pre-allocating the entire GPU memory at the start of program execution.
# This option allows for better monitoring of system resource utilization.
physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
# ===============================================================================================================
# This function creates training, validation and test datasets using the file structure created in the
# 01_create_train_val_test_directories notebook. 
# ===============================================================================================================
def create_tensorflow_datasets(image_size, train_directory, val_directory, test_directory, batch_size=32):
    
    train_dataset = image_dataset_from_directory(directory = train_directory,
                                                 labels='inferred',
                                                 label_mode = 'int',
                                                 image_size=image_size,
                                                 batch_size=batch_size,
                                                 smart_resize=True)

    val_dataset = image_dataset_from_directory(directory = val_directory,
                                               labels='inferred',
                                               label_mode = 'int',
                                               image_size=image_size,
                                               batch_size=batch_size,
                                               smart_resize=True)

    test_dataset = image_dataset_from_directory(directory = test_directory,
                                                labels = "inferred",
                                                label_mode = "int",
                                                image_size=image_size,
                                                batch_size=batch_size,
                                                smart_resize=True)
    
    return train_dataset, val_dataset, test_dataset

In [4]:
train_dataset, val_dataset, test_dataset = create_tensorflow_datasets(image_size=(520, 520),
                                                                      train_directory=train_directory,
                                                                      val_directory=val_directory,
                                                                      test_directory=test_directory,
                                                                      batch_size=32)

Found 10520 files belonging to 196 classes.
Found 3234 files belonging to 196 classes.
Found 2431 files belonging to 196 classes.


In [5]:
model_filepath = "./trained_models/convnet/2021_07_24-09_17_59_LR_DECAY_Cyclic_Decay_FINE_TUNE1_HALF_BLOCK5.keras"

model = load_model(model_filepath)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 520, 520, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 520, 520, 3)       0         
_________________________________________________________________
resnet101v2 (Functional)     (None, 17, 17, 2048)      42626560  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0     

In [7]:
'''
model_filepath = "./trained_models/convnet/2021_07_24-09_17_59_LR_DECAY_Cyclic_Decay_FINE_TUNE1_HALF_BLOCK5.keras"

model = load_model(model_filepath)

model.layers[4].trainable = True

block_5_layers = [343, 347, 350, 351, 355, 359, 362, 366, 370, 373]
block_5_layers_top_half = [359, 362, 366, 370, 373]
block_5_layers_bottom_half = [343, 347, 350, 351, 355]

layers_to_freeze = [index for index, val in enumerate(model.layers[4].layers)]

layers_to_freeze = [layer for layer in layers_to_freeze if layer not in block_5_layers_top_half]

# Make the top half of block 5 trainable
for layer_num in layers_to_freeze:
    model.layers[4].layers[layer_num].trainable = False

for index, layer_num in enumerate(block_5_layers):
    print("================================")
    print(f"Layer Number: {index}")
    print(f"Layer: {layer_num}")
    print(f"Resnet Layer Num: {model.layers[4].layers[layer_num]}")
    print(f"Num Trainable Weights: {len(model.layers[4].layers[layer_num].trainable_weights)}")
    print("================================\n")''';

In [8]:
model.layers[4].summary()

Model: "resnet101v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 520, 520, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 526, 526, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 260, 260, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 262, 262, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [9]:
model_save_path = "./trained_models/convnet/2021_07_24-09_17_59_FINE_TUNE_2_HALF_BLOCK5_EPOCHS_15_50.keras"

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                 save_best_only=True,
                                                 monitor="val_loss",
                                                 verbose=1),
             keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                               factor=0.5,
                                               patience=5,
                                               min_lr=1e-6,
                                               verbose=1)]

model.compile(loss = SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=8e-6),
              metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 520, 520, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 520, 520, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 520, 520, 3)       0         
_________________________________________________________________
resnet101v2 (Functional)     (None, 17, 17, 2048)      42626560  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0     

In [10]:
history = model.fit(train_dataset,
                    epochs=35,
                    validation_data=val_dataset,
                    callbacks=callbacks)

Epoch 1/35
329/329 [==============================] - 248s 732ms/step - loss: 1.1882 - accuracy: 0.6527 - val_loss: 1.1125 - val_accuracy: 0.6793

Epoch 00001: val_loss improved from inf to 1.11246, saving model to ./trained_models/convnet\2021_07_24-09_17_59_FINE_TUNE_2_HALF_BLOCK5_EPOCHS_15_50.keras


C:\Users\Braden\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/35
329/329 [==============================] - 240s 730ms/step - loss: 1.1737 - accuracy: 0.6467 - val_loss: 1.0914 - val_accuracy: 0.6865

Epoch 00002: val_loss improved from 1.11246 to 1.09135, saving model to ./trained_models/convnet\2021_07_24-09_17_59_FINE_TUNE_2_HALF_BLOCK5_EPOCHS_15_50.keras
Epoch 3/35
329/329 [==============================] - 239s 725ms/step - loss: 1.1602 - accuracy: 0.6538 - val_loss: 1.0842 - val_accuracy: 0.6874

Epoch 00003: val_loss improved from 1.09135 to 1.08416, saving model to ./trained_models/convnet\2021_07_24-09_17_59_FINE_TUNE_2_HALF_BLOCK5_EPOCHS_15_50.keras
Epoch 4/35
329/329 [==============================] - 239s 724ms/step - loss: 1.1327 - accuracy: 0.6668 - val_loss: 1.0752 - val_accuracy: 0.6880

Epoch 00004: val_loss improved from 1.08416 to 1.07522, saving model to ./trained_models/convnet\2021_07_24-09_17_59_FINE_TUNE_2_HALF_BLOCK5_EPOCHS_15_50.keras
Epoch 5/35
329/329 [==============================] - 239s 724ms/step - loss: 1

In [11]:
df = pd.DataFrame(history.history)

df.to_csv("./model_histories/2021_07_24-09_17_59_FINE_TUNE1_HALF_BLOCK5_EPOCHS_15_to_50.csv", index = False)

In [12]:
df.head()

,loss,accuracy,val_loss,val_accuracy,lr
0,1.188186,0.652662,1.112460,0.679344,0.000008
1,1.173671,0.646673,1.091355,0.686456,0.000008
2,1.160191,0.653802,1.084157,0.687384,0.000008
3,1.132680,0.666825,1.075222,0.688002,0.000008
4,1.137451,0.660741,1.059628,0.696660,0.000008
